# Find branch years

### for experiments where I have made a global annual average

In [1]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import xarray as xr
import intake

col_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(col_url)

In [142]:
# check which folders are available, and make a set of available experiments

experiments = {}

directory = '../Processed_data/Global_annual_means/'

model_names = [ f.name for f in os.scandir(directory) if f.is_dir() and f.name !='.ipynb_checkpoints']

for model in model_names:
    #print(model)
    experiments[model] = {}
    modeldirectory = os.path.join(directory, model)
    modelexp_names = [ f.name for f in os.scandir(modeldirectory) if f.is_dir() and f.name !='.ipynb_checkpoints']
    for exp in modelexp_names:
        #print(exp)
        experiments[model][exp] = {}
        modelexpdirectory = os.path.join(modeldirectory, exp)
        filenames = [f.name for f in os.scandir(modelexpdirectory) if f.name !='.ipynb_checkpoints']
        
        members = [file.rsplit('_')[2] for file in filenames]
        members.sort()
        experiments[model][exp] = members

#experiments   

## Select model, and choose whether to make a completely new file or add new info to existing file

In [133]:
#model = 'UKESM1-0-LL'
model = 'FGOALS-g3'
#model = 'EC-Earth3-Veg-LR'

newfile = False

experiments[model]

{'ssp585': ['r1i1p1f1'],
 'piControl': ['r1i1p1f1'],
 'abrupt-4xCO2': ['r1i1p1f1'],
 'ssp245': ['r1i1p1f1'],
 'historical': ['r1i1p1f1',
  'r2i1p1f1',
  'r3i1p1f1',
  'r4i1p1f1',
  'r5i1p1f1',
  'r6i1p1f1'],
 '1pctCO2': ['r1i1p1f1', 'r2i1p1f1', 'r3i1p1f1'],
 'ssp370': ['r1i1p1f1'],
 'ssp126': ['r1i1p1f1']}

In [134]:
filename = '../Processed_data/Branch_times/' + model + '_branch_times.txt'
column_names = ['model', 'exp', 'member', 'parent_experiment_id', 'parent_variant_id', 
                'branch_time_in_child', 'branch_time_in_parent', 'parent_time_units', 'branch_method']
df = pd.DataFrame(columns = column_names)

if newfile == True:
    modelexp_toload = experiments[model]
else:
    # check if branch info is already printed, and if so, reuse existing info and load only new experiments
    if os.path.isfile(filename):
        previously_read = pd.read_table(filename,index_col=0, sep = ' ')
        modelexp_toload = {}    

        # does it cover all available experiments and members?
        for exp in experiments[model]:
            if exp in previously_read['exp'].values:  # then at least one member has been loaded before
                previously_read_exp = previously_read[exp == previously_read['exp']] # dataframe reduced to only this experiment
                for member in experiments[model][exp]:
                    if member in previously_read_exp['member'].values:
                        print(exp, member, ' exists')
                    else:
                        if exp not in modelexp_toload:
                            modelexp_toload[exp] = [member] # needs to be tested for a model with new members
                        else:
                            modelexp_toload[exp].append(member)
                        print('other members of', exp, 'have been looked at before, but member ', member, 'needs to be loaded')
            else:
                print(exp,  ' is not available in file and needs to be loaded for all members', experiments[model][exp])
                modelexp_toload[exp] = experiments[model][exp]
    else:
        print('no file exists already, so a new file will be created and all data will be loaded')
        newfile = True
        modelexp_toload = experiments[model]
        #print(modelexp_toload)


print('\n', 'Scenarios and members to be loaded:')
modelexp_toload           

no file exists already, so a new file will be created and all data will be loaded

 Scenarios and members to be loaded:


{'ssp585': ['r1i1p1f1'],
 'piControl': ['r1i1p1f1'],
 'abrupt-4xCO2': ['r1i1p1f1'],
 'ssp245': ['r1i1p1f1'],
 'historical': ['r1i1p1f1',
  'r2i1p1f1',
  'r3i1p1f1',
  'r4i1p1f1',
  'r5i1p1f1',
  'r6i1p1f1'],
 '1pctCO2': ['r1i1p1f1', 'r2i1p1f1', 'r3i1p1f1'],
 'ssp370': ['r1i1p1f1'],
 'ssp126': ['r1i1p1f1']}

In [135]:
#for model in experiments:
#for exp in ['piClim-control', 'piControl', 'abrupt-4xCO2', 'piClim-4xCO2']:
for exp in modelexp_toload:
    for member in modelexp_toload[exp]:

        cat = col.search(experiment_id = exp, variable_id='rlut', table_id='Amon', source_id = model, member_id = member)
        dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})
        for key in dset_dict.keys():
            ds = dset_dict[key]
        parent_experiment_id = ds.parent_experiment_id
        parent_variant_label = ds.parent_variant_label
        branch_time_in_child = ds.branch_time_in_child
        branch_time_in_parent = ds.branch_time_in_parent
        parent_time_units = ds.parent_time_units
        branch_method = ds.branch_method
        if model == 'EC-Earth3':
            branch_time_in_child = str(branch_time_in_child).replace('D', '')
            branch_time_in_parent = str(branch_time_in_parent).replace('D', '')

        print(model, exp, member, parent_experiment_id, parent_variant_label, branch_time_in_child, branch_time_in_parent, parent_time_units, branch_method)
        df_row = pd.DataFrame([[model, exp, member, parent_experiment_id, parent_variant_label,
                   branch_time_in_child, branch_time_in_parent, parent_time_units, branch_method]] ,columns = column_names)
        df = df.append(df_row, ignore_index = True)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


FGOALS-g3 ssp585 r1i1p1f1 historical r1i1p1f1 0.0 60225.0 days since 1850-01-01 standard

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


FGOALS-g3 piControl r1i1p1f1 piControl-spinup r1i1p1f1 0.0 0.0 days since 1850-01-01 standard

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


FGOALS-g3 abrupt-4xCO2 r1i1p1f1 piControl r1i1p1f1 0.0 168630.0 days since 0001-01-01 standard

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


FGOALS-g3 ssp245 r1i1p1f1 historical r1i1p1f1 0.0 60225.0 days since 1850-01-01 standard

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


...

FGOALS-g3 1pctCO2 r1i1p1f1 piControl r1i1p1f1 0.0 251485.0 days since 0001-01-01 standard

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


FGOALS-g3 1pctCO2 r2i1p1f1 piControl r1i1p1f1 0.0 251485.0 days since 0001-01-01 standard

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


FGOALS-g3 1pctCO2 r3i1p1f1 piControl r1i1p1f1 0.0 251485.0 days since 0001-01-01 standard

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


FGOALS-g3 ssp370 r1i1p1f1 historical r1i1p1f1 0.0 60225.0 days since 1850-01-01 standard

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


FGOALS-g3 ssp126 r1i1p1f1 historical r1i1p1f1 0.0 60225.0 days since 1850-01-01 standard


In [140]:
df

,model,exp,member,parent_experiment_id,parent_variant_id,branch_time_in_child,branch_time_in_parent,parent_time_units,branch_method
0,FGOALS-g3,1pctCO2,r1i1p1f1,piControl,r1i1p1f1,0.0,251485.0,days since 0001-01-01,standard
1,FGOALS-g3,1pctCO2,r2i1p1f1,piControl,r1i1p1f1,0.0,251485.0,days since 0001-01-01,standard
2,FGOALS-g3,1pctCO2,r3i1p1f1,piControl,r1i1p1f1,0.0,251485.0,days since 0001-01-01,standard
3,FGOALS-g3,abrupt-4xCO2,r1i1p1f1,piControl,r1i1p1f1,0.0,168630.0,days since 0001-01-01,standard
4,FGOALS-g3,historical,r1i1p1f1,piControl,r1i1p1f1,0.0,134685.0,days since 0001-01-01,standard
5,FGOALS-g3,historical,r2i1p1f1,piControl,r1i1p1f1,0.0,134685.0,days since 0001-01-01,standard
6,FGOALS-g3,historical,r3i1p1f1,piControl,r1i1p1f1,0.0,134685.0,days since 0001-01-01,standard
7,FGOALS-g3,historical,r4i1p1f1,piControl,r1i1p1f1,0.0,134685.0,days since 0001-01-01,standard
8,FGOALS-g3,historical,r5i1p1f1,piControl,r1i1p1f1,0.0,134685.0,days since 0001-01-01,standard
9,FGOALS-g3,historical,r6i1p1f1,piControl,r1i1p1f1,0.0,134685.0,days since 0001-01-01,standard


In [141]:
if newfile == True:
    df = df.sort_values(by=['exp','member'], ignore_index = True)
    df.to_csv(filename, ' ')
else:
    # merge with previously made file before saving
    new_df = pd.concat([previously_read, df], ignore_index = True).sort_values(by=['exp','member'], ignore_index = True)
    new_df.to_csv(filename, ' ')

## Run through all models:

In [147]:
len(model_names)

56

In [146]:
for model in model_names:
    print(model)
    newfile = False
    filename = '../Processed_data/Branch_times/' + model + '_branch_times.txt'
    column_names = ['model', 'exp', 'member', 'parent_experiment_id', 'parent_variant_id', 
                    'branch_time_in_child', 'branch_time_in_parent', 'parent_time_units', 'branch_method']
    df = pd.DataFrame(columns = column_names)

    if newfile == True:
        modelexp_toload = experiments[model]
    else:
        # check if branch info is already printed, and if so, reuse existing info and load only new experiments
        if os.path.isfile(filename):
            previously_read = pd.read_table(filename,index_col=0, sep = ' ')
            modelexp_toload = {}    

            # does it cover all available experiments and members?
            for exp in experiments[model]:
                if exp in previously_read['exp'].values:  # then at least one member has been loaded before
                    previously_read_exp = previously_read[exp == previously_read['exp']] # dataframe reduced to only this experiment
                    for member in experiments[model][exp]:
                        if member in previously_read_exp['member'].values:
                            print(exp, member, ' exists')
                        else:
                            if exp not in modelexp_toload:
                                modelexp_toload[exp] = [member] # needs to be tested for a model with new members
                            else:
                                modelexp_toload[exp].append(member)
                            print('other members of', exp, 'have been looked at before, but member ', member, 'needs to be loaded')
                else:
                    print(exp,  ' is not available in file and needs to be loaded for all members', experiments[model][exp])
                    modelexp_toload[exp] = experiments[model][exp]
        else:
            print('no file exists already, so a new file will be created and all data will be loaded')
            newfile = True
            modelexp_toload = experiments[model]
            #print(modelexp_toload)


    print('\n', 'Scenarios and members to be loaded:')
    print(modelexp_toload)       
          

    for exp in modelexp_toload:
        for member in modelexp_toload[exp]:

            cat = col.search(experiment_id = exp, variable_id='rlut', table_id='Amon', source_id = model, member_id = member)
            dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})
            for key in dset_dict.keys():
                ds = dset_dict[key]
            parent_experiment_id = ds.parent_experiment_id
            parent_variant_label = ds.parent_variant_label
            branch_time_in_child = ds.branch_time_in_child
            branch_time_in_parent = ds.branch_time_in_parent
            parent_time_units = ds.parent_time_units
            branch_method = ds.branch_method
            if model == 'EC-Earth3':
                branch_time_in_child = str(branch_time_in_child).replace('D', '')
                branch_time_in_parent = str(branch_time_in_parent).replace('D', '')

            print(model, exp, member, parent_experiment_id, parent_variant_label, branch_time_in_child, branch_time_in_parent, parent_time_units, branch_method)
            df_row = pd.DataFrame([[model, exp, member, parent_experiment_id, parent_variant_label,
                       branch_time_in_child, branch_time_in_parent, parent_time_units, branch_method]] ,columns = column_names)
            df = df.append(df_row, ignore_index = True)
            
    if newfile == True:
        df = df.sort_values(by=['exp','member'], ignore_index = True)
        df.to_csv(filename, ' ')
    else:
        # merge with previously made file before saving
        new_df = pd.concat([previously_read, df], ignore_index = True).sort_values(by=['exp','member'], ignore_index = True)
        new_df.to_csv(filename, ' ')

MRI-ESM2-0
piClim-control r1i1p1f1  exists
ssp585 r1i2p1f1  exists
abrupt-2xCO2 r1i1p1f1  exists
abrupt-0p5xCO2 r1i1p1f1  exists
piControl r1i1p1f1  exists
piControl r1i1p1f1  exists
piControl r1i2p1f1  exists
abrupt-4xCO2 r10i1p1f1  exists
abrupt-4xCO2 r11i1p1f1  exists
abrupt-4xCO2 r12i1p1f1  exists
abrupt-4xCO2 r13i1p1f1  exists
abrupt-4xCO2 r1i1p1f1  exists
abrupt-4xCO2 r1i1p1f1  exists
abrupt-4xCO2 r1i2p1f1  exists
abrupt-4xCO2 r2i1p1f1  exists
abrupt-4xCO2 r3i1p1f1  exists
abrupt-4xCO2 r4i1p1f1  exists
abrupt-4xCO2 r5i1p1f1  exists
abrupt-4xCO2 r6i1p1f1  exists
abrupt-4xCO2 r7i1p1f1  exists
abrupt-4xCO2 r8i1p1f1  exists
abrupt-4xCO2 r9i1p1f1  exists
ssp245 r1i1p1f1  exists
other members of ssp245 have been looked at before, but member  r1i3p1f1 needs to be loaded
ssp245 r2i1p1f1  exists
other members of ssp245 have been looked at before, but member  r2i3p1f1 needs to be loaded
ssp245 r3i1p1f1  exists
other members of ssp245 have been looked at before, but member  r3i3p1f1 needs t

MRI-ESM2-0 1pctCO2 r1i1p1f1 piControl r1i1p1f1 0.0 0.0 days since 1850-01-01 standard

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


MRI-ESM2-0 1pctCO2 r1i2p1f1 piControl r1i2p1f1 0.0 0.0 days since 1850-01-01 standard
SAM0-UNICON
no file exists already, so a new file will be created and all data will be loaded

 Scenarios and members to be loaded:
{'piControl': ['r1i1p1f1'], 'abrupt-4xCO2': ['r1i1p1f1'], 'historical': ['r1i1p1f1'], '1pctCO2': ['r1i1p1f1']}

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


SAM0-UNICON piControl r1i1p1f1 piControl-spinup r1i1p1f1 0.0 19710.0 days since 0001-01-01 standard

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


SAM0-UNICON abrupt-4xCO2 r1i1p1f1 piControl r1i1p1f1 0.0 99645.0 days since 0001-01-01 standard

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


...

NESM3 historical r5i1p1f1 piControl r1i1p1f1 0.0 54787.0 days since 1000-01-01 00:00:00 standard

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


NESM3 1pctCO2 r1i1p1f1 piControl r1i1p1f1 0.0 54787.0 days since 1000-01-01 00:00:00 standard

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


NESM3 ssp126 r1i1p1f1 historical r1i1p1f1 0.0 2015.0 days since 1850-01-01 00:00:00 standard

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


NESM3 ssp126 r2i1p1f1 historical r2i1p1f1 0.0 2015.0 days since 1850-01-01 00:00:00 standard
MIROC-ES2L
ssp585 r1i1p1f2  exists
piControl r1i1p1f2  exists
abrupt-4xCO2 r1i1p1f2  exists
ssp245 r1i1p1f2  exists
historical r10i1p1f2  exists
historical r1i1p1f2  exists
historical r2i1p1f2  exists
historical r3i1p1f2  exists
historical r4i1p1f2  exists
historical r5i1p1f2  exists
historical r6i1p1f2  exists
historical r7i1p1f2  exists
historical r8i1p1f2  exists
historical r9i1p1f2  exists
1pctCO2  is not available in file and needs to be loaded for all members ['r1i1p1f2']
ssp370 r1i1p1f2  exists
ssp126 r1i1p1f2  exists
ssp126 r2i1p1f2  exists
ssp126 r3i1p1f2  exists

 Scenarios and members to be loaded:
{'1pctCO2': ['r1i1p1f2']}

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


MIROC-ES2L 1pctCO2 r1i1p1f2 piControl r1i1p1f2 0.0 0.0 days since 1850-1-1 standard
